__Assignment 1 __

1) What are the lowest and highest temperatures measured each year for the period 1950- 2014. Provide the lists sorted in the descending order with respect to the maximum temperature. In this exercise you will use the temperature-readings.csv file.

In [1]:
# create the setup
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
sqlContext=SQLContext(sc)


In [10]:
path_unifolder_temp_tiny = "/Users/phillipholscher/OneDrive - Linköpings universitet/LiU/Statistics and Machine Learning/Spring19T2/Big Data Analytics/Labs/Lab 2/temperature-readings-tiny.csv"


In [5]:
# read and create rdd, this needs to be tranfsormed to data frame later on
temps= sc.textFile(path_unifolder_temp_tiny)
parts = temps.map(lambda l:l.split(";"))
tempReadings = parts.map(lambda p: Row(station=p[0],  date=p[1], year=p[1].split("-")[0], time=p[2],  Temp=float(p[3]), quality=p[4]))


In [6]:
tempReadings.take(10)

[Row(Temp=6.8, date='2013-11-01', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=3.8, date='2013-11-01', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=5.8, date='2013-11-02', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=-1.1, date='2013-11-02', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=-0.2, date='2013-11-03', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=5.6, date='2013-11-03', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=6.5, date='2013-11-04', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=5.1, date='2013-11-04', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=4.2, date='2013-11-05', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=3.2, date='2013-11-05', quality='G', station='102170', time='18:00:00', year='2013')]

In [7]:
# create a data frame from rdd
schemaTempReadings =  sqlContext.createDataFrame(tempReadings)
schemaTempReadings.registerTempTable("tempReadings")
schemaTempReadings.show(10)

In [9]:
schemaTempReadings.show(10)

+----+----------+-------+-------+--------+----+
|Temp|      date|quality|station|    time|year|
+----+----------+-------+-------+--------+----+
| 6.8|2013-11-01|      G| 102170|06:00:00|2013|
| 3.8|2013-11-01|      G| 102170|18:00:00|2013|
| 5.8|2013-11-02|      G| 102170|06:00:00|2013|
|-1.1|2013-11-02|      G| 102170|18:00:00|2013|
|-0.2|2013-11-03|      G| 102170|06:00:00|2013|
| 5.6|2013-11-03|      G| 102170|18:00:00|2013|
| 6.5|2013-11-04|      G| 102170|06:00:00|2013|
| 5.1|2013-11-04|      G| 102170|18:00:00|2013|
| 4.2|2013-11-05|      G| 102170|06:00:00|2013|
| 3.2|2013-11-05|      G| 102170|18:00:00|2013|
+----+----------+-------+-------+--------+----+
only showing top 10 rows



In [33]:
# we want to know the max and min temp for a specifitc time interval 
# time interval >= 1950 & <= 2014
schemaTempReadings_filterd = schemaTempReadings.select(["Temp","year"]).filter((schemaTempReadings["year"] >= 1950) &  
                                                                          (schemaTempReadings["year"] <= 2014))
schemaTempReadings_filterd.show(10)


+----+----+
|Temp|year|
+----+----+
| 6.8|2013|
| 3.8|2013|
| 5.8|2013|
|-1.1|2013|
|-0.2|2013|
| 5.6|2013|
| 6.5|2013|
| 5.1|2013|
| 4.2|2013|
| 3.2|2013|
+----+----+
only showing top 10 rows



In [48]:
# get the max temp for every year
schemaTempReadings_max = schemaTempReadings_filterd.groupBy(["year"]).agg(F.max("Temp").alias("Temp_max"))
schemaTempReadings_max.show(10)

+----+--------+
|year|Temp_max|
+----+--------+
|1957|    25.2|
|1956|    26.0|
|1958|    28.1|
|2014|    29.1|
|2013|    10.2|
|1959|    28.2|
|1961|    19.0|
|1955|    20.4|
|1960|    29.0|
+----+--------+



In [51]:
# order the temp with respect to the maximum temperature
schemaTempReadings_max_orderd = schemaTempReadings_max.sort("Temp_max", ascending = False)
schemaTempReadings_max_orderd.show(10)

+----+--------+
|year|Temp_max|
+----+--------+
|2014|    29.1|
|1960|    29.0|
|1959|    28.2|
|1958|    28.1|
|1956|    26.0|
|1957|    25.2|
|1955|    20.4|
|1961|    19.0|
|2013|    10.2|
+----+--------+



In [54]:
# now we do the same for min temp
schemaTempReadings_min = schemaTempReadings_filterd.groupBy(["year"]).agg(F.min("Temp").alias("Temp_min"))\
.sort("Temp_min", ascending = True)
schemaTempReadings_min.show(10)

+----+--------+
|year|Temp_min|
+----+--------+
|1956|   -30.0|
|1960|   -28.3|
|1958|   -27.9|
|1955|   -26.2|
|2014|   -24.3|
|1961|   -23.5|
|1959|   -23.2|
|1957|   -19.9|
|2013|   -13.3|
+----+--------+

